In [51]:
import numpy as np
from scipy.ndimage import rotate


In [101]:
p_input = """.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
.........."""

In [102]:
with open("4-1.txt") as f:
    p_input = f.read()

First reaction is `D:`

Ok so this doesn't seem _that_ different - we're still looking for the string "MAS", but it seems like the "A" needs to be part of 2 "MAS"es. Instict is saying we could keep track of the cordinates of each Ain each directino (e.g NW-SE, NE-SW), and if that coordinate is in the list for both then it's the center of a valid X-MAS.

Ok so essentially we keep track of how many "MAS"es cross over each "A". What if there's 4 MASes, is that 1 cross or 2? The specification doesn't say - let's say it's 2 for now. Either way, we should group the searches: diagnoals in the bottom left-top right direction (either way) can only cross with diagnolas in the transpose for instance.

In [24]:
matches = 0
p_list = np.array([list(line) for line in p_input.split()], dtype=str)

hor_coords = [(i, j+2) for i, line in enumerate(p_list) for j in range(len(line)) 
           if "".join(line)[j:j+4] == "XMAS"]

horb_coords = [(i, j+1) for i, line in enumerate(p_list) for j in range(len(line)) 
           if "".join(line)[j:j+4] == "SAMX"]

ver_coords = [(i, j+2) for i, line in enumerate(p_list.T) for j in range(len(line)) 
           if "".join(line)[j:j+4] == "XMAS"]

verb_coords = [(i, j+1) for i, line in enumerate(p_list.T) for j in range(len(line)) 
           if "".join(line)[j:j+4] == "SAMX"]


Ugh. I should have stared at the examples harder - X-es are _literally_ X-es, not crosses. That actually makes this a lot easier.

Sketching this out on paper. There's clearly a relationship between the k value, the position and which "MAS" is found, and it's coordinates.

...

Ok I spent over an hour putting together a coordinate system, which I'll summarise with,


**A Brief Treatsie on an Unnecessary Coordinate Transformation**

Consider a diagonal grid system where we measure positions using two parameters:

- $K$: representing distance from a central diagonal, top-right to bottom-left, centre diagonal numbered $0$ and increasing in a negative $y$ direction
- $L$: measuring a position along diagonals parallel to the central line, increasing top-right to bottom-left

We can define a transformation $T: ℝ² → ℝ²$ that maps these coordinates to a normalized diagonal basis:

$$
T: (K,L) ↦ (K̂,L̂)
$$

Where:

$$
\hat{K} = K + max(K)\\
\hat{L} = L + abs(K)
$$


This transformation effectively normalises our asymmetric diagonal measurements into a regular coordinate system rotated 135° clockwise from standard Cartesian coordinates. The resulting $(K̂,L̂)$ space exhibits elegant symmetry properties, with $K̂$ increasing along the top-right to bottom-left diagonal and $L̂$ increasing along the top-left to bottom-right diagonal.

This was all useless because we can just check all the points around a given "A"

In [105]:
matches = 0
p_list = np.array([list(line) for line in p_input.split()], dtype=str)

max_line, max_col = len(p_list), len(p_list[0])
targets = ("MAS", "SAM")
total = 0

for line_n, line in enumerate(p_list):
    for col_n, char in enumerate(line):
        if 0 < line_n < max_line-1 and 0 < col_n < max_col-1:
            if char == "A":
                nw = p_list[line_n-1][col_n-1]
                ne = p_list[line_n-1][col_n+1]
                sw = p_list[line_n+1][col_n-1]
                se = p_list[line_n+1][col_n+1]

                if "".join([nw, char, se]) in targets and "".join([ne, char, sw]) in targets:
                    total += 1

print(total)

1871
